# DataAnalysis

## Paquetes

In [60]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

## Funciones

In [61]:
def extract_coordinates(point):
    point = point.strip("()")
    x, y = point.split(", ")
    return int(x), int(y)

def type_point(row):
    # empty
    if 0 in row.values:
        return 0
    # feeding
    elif -2 in row.values:
        return -2
    else:
        return 1

## Código

### Lectura de datos

In [62]:
# Read the data from the excel file
data = pd.read_excel('data/output_points_6_no.xlsx')

# Split columns into two
for column in data.columns:
    data[column + ' (x)'] = data[column].apply(lambda p: extract_coordinates(p)[0])
    data[column + ' (y)'] = data[column].apply(lambda p: extract_coordinates(p)[1])

# Drop the original columns
data = data.drop(columns=data.columns[:6])

# Delete column Right Point 1 y 2 (x)
data = data.drop(columns=['Right Point 1 (x)', 'Right Point 2 (x)'])

# Add type column
data['type'] = data.apply(type_point, axis=1)

# Delete rows with type 0 values
data = data[data['type'] != 0]
data = data.reset_index(drop=True)

# Change -1 values to nan
data = data.replace(-1, np.nan)

# Find first row with the value equal to -2
feed_o = data.index[data['type'] == -2].tolist()[0]
# Find last row with the value equal to -2
feed_f = data.index[data['type'] == -2].tolist()[-1]

# Separate feeding points
data_bef = data.iloc[:feed_o]
data_fee = data.iloc[feed_o:feed_f+1]
data_aft = data.iloc[feed_f+1:]

# Complete nan by interpolation
data_bef = data_bef.interpolate()
data_fee = data_fee.interpolate()
data_aft = data_aft.interpolate()

# Recalculate Right Point 1 (y) and Right Point 2 (y)
data_bef['Left Point 1 (z)'] = 480 - data_bef['Right Point 1 (y)']
data_bef['Left Point 2 (z)'] = 480 - data_bef['Right Point 2 (y)']
data_fee['Left Point 1 (z)'] = 480 - data_fee['Right Point 1 (y)']
data_fee['Left Point 2 (z)'] = 480 - data_fee['Right Point 2 (y)']
data_aft['Left Point 1 (z)'] = 480 - data_aft['Right Point 1 (y)']
data_aft['Left Point 2 (z)'] = 480 - data_aft['Right Point 2 (y)']

In [63]:
# Take Left Point 1 and 2 and Right Point a 1 and 2
flower_bef = data_bef[['Left Point 1 (x)', 'Left Point 1 (y)', 'Left Point 1 (z)']]
flower_fee = data_fee[['Left Point 1 (x)', 'Left Point 1 (y)', 'Left Point 1 (z)']]
flower_aft = data_aft[['Left Point 1 (x)', 'Left Point 1 (y)', 'Left Point 1 (z)']]

bat_bef = data_bef[['Left Point 2 (x)', 'Left Point 2 (y)', 'Left Point 2 (z)']]
bat_fee = flower_fee
bat_aft = data_aft[['Left Point 2 (x)', 'Left Point 2 (y)', 'Left Point 2 (z)']]

# Rename columns
flower_bef.columns = ['x', 'y', 'z']
flower_fee.columns = ['x', 'y', 'z']
flower_aft.columns = ['x', 'y', 'z']

bat_bef.columns = ['x', 'y', 'z']
bat_fee.columns = ['x', 'y', 'z']
bat_aft.columns = ['x', 'y', 'z']

# Show the data
print(bat_fee.head())

# Concatenate the data
flower = pd.concat([flower_bef, flower_fee, flower_aft])
bat = pd.concat([bat_bef, bat_fee, bat_aft])
data = pd.concat([flower, bat])

        x      y      z
12  264.0  109.0  401.0
13  265.0  107.0  400.0
14  264.0   95.0  395.0
15  257.0   92.0  384.0
16  252.0   93.0  356.0


### Gráfico 3D

In [64]:
df = flower_fee.copy()

In [65]:
# Crear el gráfico 3D interactivo
fig = go.Figure()

# Añadir la trayectoria con todos los puntos menos el último
fig.add_trace(go.Scatter3d(
    x=df['x'][:-1],
    y=df['y'][:-1],
    z=df['z'][:-1],
    mode='lines+markers',
    marker=dict(size=2),
    line=dict(width=2),
    name = 'Trayectoria'
))

# Configurar el diseño del gráfico
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(showticklabels=False, tickmode="linear", dtick=50, range=[min(df['x']) - 10, max(df['x']) + 10], backgroundcolor="lightblue", gridwidth=0.5),
        yaxis=dict(showticklabels=False, tickmode="linear", dtick=50, range=[min(df['y']) - 10, max(df['y']) + 10], backgroundcolor="lightblue", gridwidth=0.5),
        zaxis=dict(showticklabels=False, tickmode="linear", dtick=50,  range=[min(df['z']) - 10, max(df['z']) + 10], backgroundcolor="lightblue", gridwidth=0.5)
    ),
    title='Trayectoria 3D Interactiva'
)

# Configurar la cámara para que salga cuadrado y alejado de los puntos
camera = dict(
    eye=dict(x=1.5, y=1.5, z=1.5)
)

fig.update_layout(scene_camera=camera) 

# Mostrar el gráfico
fig.show()

C:\Users\aemer\AppData\Local\Temp\ipykernel_24256\2118972295.py:6: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

C:\Users\aemer\AppData\Local\Temp\ipykernel_24256\2118972295.py:7: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

C:\Users\aemer\AppData\Local\Temp\ipykernel_24256\2118972295.py:8: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` 

### Curvatura

In [66]:
# Convertir a arrays de numpy
x = np.array(df['x'])
y = np.array(df['y'])
z = np.array(df['z'])

# Calcular las derivadas usando diferencias finitas
dx = np.gradient(x)
dy = np.gradient(y)
dz = np.gradient(z)

ddx = np.gradient(dx)
ddy = np.gradient(dy)
ddz = np.gradient(dz)

# Calcular la curvatura
curvatura = np.zeros(len(x))

for i in range(len(x)):
    # Vectores de la primera y segunda derivada
    r_prime = np.array([dx[i], dy[i], dz[i]])
    r_double_prime = np.array([ddx[i], ddy[i], ddz[i]])
    
    # Producto cruzado de r' y r''
    cross_product = np.cross(r_prime, r_double_prime)
    
    # Magnitud de r'
    norm_r_prime = np.linalg.norm(r_prime)
    
    # Curvatura
    curvatura[i] = np.linalg.norm(cross_product) / (norm_r_prime**3)

# Añadir la curvatura al DataFrame
df['curvatura'] = curvatura

# Desviación Estánda de la curvatura
std_curvatura = np.std(curvatura)

# Mostrar la desviación estándar de la curvatura
print('Desviación Estándar de la Curvatura:', std_curvatura)


# Mostrar el DataFrame
print(df.head(20))

Desviación Estándar de la Curvatura: 0.11839630768194657
        x      y      z  curvatura
12  264.0  109.0  401.0   0.522636
13  265.0  107.0  400.0   0.056668
14  264.0   95.0  395.0   0.058465
15  257.0   92.0  384.0   0.018195
16  252.0   93.0  356.0   0.016047
17  247.0  106.0  346.0   0.013642
18  245.0  115.0  332.0   0.005260
19  240.0  124.0  326.0   0.021481
20  238.0  127.0  315.0   0.028453
21  235.0  128.0  308.0   0.035979
22  235.0  128.0  301.0   0.080051
23  232.0  124.0  298.0   0.056583
24  230.0  121.0  293.0   0.056258
25  229.0  126.0  283.0   0.005709
26  227.0  128.0  277.0   0.139049
27  226.0  127.0  274.0   0.052802
28  223.0  122.0  271.0   0.080044
29  224.0  122.0  264.0   0.137986
30  224.0  126.0  264.0   0.081246
31  220.0  126.0  256.0   0.303592


### Animación

In [67]:
df = bat.copy()

In [79]:
# Crear el gráfico 3D interactivo
fig = go.Figure(
    data=[
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(),
        go.Scatter3d(
            x=[bat['x'].iloc[0]],
            y=[bat['y'].iloc[0]],
            z=[bat['z'].iloc[0]],
            mode='markers',
            marker=dict(size=5, color='black'), 
            name='Murciélago'
        ),
        go.Scatter3d(
            x=bat['x'][:-1],
            y=bat['y'][:-1],
            z=bat['z'][:-1],
            mode='lines+markers',
            marker=dict(size=0.1, color='red'),
            line=dict(width=0.1),
            name='Trayectoria'
        )
    ]
)


# Agrego los frames
bef = [go.Frame(
    data=[
        go.Scatter3d(
            x=bat_bef['x'][:k+1],
            y=bat_bef['y'][:k+1],
            z=bat_bef['z'][:k+1],
            mode='lines+markers',
            marker=dict(size=[0] * (k - 1) + [5], color='black'),
            line=dict(width=4, color='black'),
            name = 'Aproximación'
        )
    ] + [
        go.Scatter3d(
            x=flower_bef['x'][:k+1],
            y=flower_bef['y'][:k+1],
            z=flower_bef['z'][:k+1],
            mode='lines+markers',
            marker=dict(size=[0] * (k - 1) + [5], color='red'),
            line=dict(width=4, color='red'),
            name = 'Flor'
        )
    ]
) for k in range(len(bat_bef))]

fee = [go.Frame(
    data=[
        go.Scatter3d(
            x=bat_fee['x'][:k+1],
            y=bat_fee['y'][:k+1],
            z=bat_fee['z'][:k+1],
            mode='lines+markers',
            marker=dict(size=[0] * (k - 1) + [5], color='blue'),
            line=dict(width=4, color='blue'),
            name = 'Alimentación'
        )
    ] + [
        go.Scatter3d(
            x=flower_bef['x'],
            y=flower_bef['y'],
            z=flower_bef['z'],
            mode='lines',
            line=dict(width=4, color='black'),
            name = 'Flor'
        )
    ] + [
        go.Scatter3d(
            x=bat_bef['x'],
            y=bat_bef['y'],
            z=bat_bef['z'],
            mode='lines',
            line=dict(width=4, color='black'),
            name = 'Aproximación'
        )
    ] + [
        go.Scatter3d(
            x=[flower_bef['x'].iloc[-1]],
            y=[flower_bef['y'].iloc[-1]],
            z=[flower_bef['z'].iloc[-1]],
            mode='markers',
            marker=dict(size=5, color='red'), 
            name='Flor'
        )
    ] + [
        go.Scatter3d(
            x=[bat_bef['x'].iloc[-1]],
            y=[bat_bef['y'].iloc[-1]],
            z=[bat_bef['z'].iloc[-1]],
            mode='markers',
            marker=dict(size=5, color='black'), 
            name='Murciélago'
        )
    ]
) for k in range(len(bat_fee))]

aft = [go.Frame(
    data=[
        go.Scatter3d(
            x=bat_aft['x'][:k+1],
            y=bat_aft['y'][:k+1],
            z=bat_aft['z'][:k+1],
            mode='lines+markers',
            marker=dict(size=[0] * (k - 1) + [5], color='black'),
            line=dict(width=4, color='black'),
            name = 'Alejamiento'
        )
    ] + [
        go.Scatter3d(
            x=flower_aft['x'][:k+1],
            y=flower_aft['y'][:k+1],
            z=flower_aft['z'][:k+1],
            mode='lines+markers',
            marker=dict(size=[0] * (k - 1) + [5], color='red'),
            line=dict(width=4, color='red'),
            name = 'Flor'
        )
    ] + [
        go.Scatter3d(
            x=[bat_fee['x'].iloc[-1]],
            y=[bat_fee['y'].iloc[-1]],
            z=[bat_fee['z'].iloc[-1]],
            mode='markers',
            marker=dict(size=5, color='black'), 
            name='Murciélago'
        )
    ] + [
        go.Scatter3d(
            x=bat_fee['x'],
            y=bat_fee['y'],
            z=bat_fee['z'],
            mode='lines',
            line=dict(width=4, color='blue'),
            name = 'Aliemntación'
        )
    ] + [
        go.Scatter3d(
            x=flower_bef['x'],
            y=flower_bef['y'],
            z=flower_bef['z'],
            mode='lines',
            line=dict(width=4, color='red'),
            name = 'Flor'
        )
    ] + [
        go.Scatter3d(
            x=bat_bef['x'],
            y=bat_bef['y'],
            z=bat_bef['z'],
            mode='lines',
            line=dict(width=4, color='black'),
            name = 'Aproximación'
        )
    ] + [
        go.Scatter3d(
            x=[flower_bef['x'].iloc[-1]],
            y=[flower_bef['y'].iloc[-1]],
            z=[flower_bef['z'].iloc[-1]],
            mode='markers',
            marker=dict(size=5, color='red'), 
            name='Flor'
        )
    ] + [
        go.Scatter3d(
            x=[bat_bef['x'].iloc[-1]],
            y=[bat_bef['y'].iloc[-1]],
            z=[bat_bef['z'].iloc[-1]],
            mode='markers',
            marker=dict(size=5, color='black'), 
            name='Murciélago'
        )
    ]
) for k in range(len(bat_aft))]

fig.frames = bef + fee + aft


# Configurar el diseño del gráfico
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        xaxis=dict(showticklabels=False, tickmode="linear", dtick=100, range=[min(data['x']) - 30, max(data['x']) + 30], backgroundcolor="lightblue", gridwidth=0.5),
        yaxis=dict(showticklabels=False, tickmode="linear", dtick=100, range=[min(data['y']) - 30, max(data['y']) + 30], backgroundcolor="lightblue", gridwidth=0.5),
        zaxis=dict(showticklabels=False, tickmode="linear", dtick=100,  range=[min(data['z']) - 30, max(data['z']) + 30], backgroundcolor="lightblue", gridwidth=0.5)
    ),
    title='Trayectoria 3D Interactiva',
    updatemenus=[dict(
            type="buttons", 
            y=1.05, x=0.8, xanchor='left', yanchor='bottom',
            buttons=[
                dict(
                    label='Play', method='animate',
                    args=[None, dict(frame=dict(duration=200, redraw=True))]
                )
            ]
    )],
)


# Mostrar el gráfico
fig.show()

C:\Users\aemer\AppData\Local\Temp\ipykernel_24256\2455947361.py:64: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

C:\Users\aemer\AppData\Local\Temp\ipykernel_24256\2455947361.py:65: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

C:\Users\aemer\AppData\Local\Temp\ipykernel_24256\2455947361.py:66: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i